In [1]:
import sys  
sys.path.insert(0, '../OMPbenders/')
%matplotlib widget

In [2]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
#from MasterProblemOMP import MasterProblem
#from SubProblemOMP import SubProblem
sys.path.insert(0, '../AuxiliarCodes/')
%matplotlib widget
from plotDrawpointsPoints import plotDrawpointsPoints
from plotGurobiOpenPitSolution import plotGurobiOpenPitSolution
from plotIntegratedOmpSolution import plotIntegratedOmpSolution
from plotOmpOpenPitSolution import plotOmpOpenPitSolution
from plotUndergroundSolution import plotUndergroundSolution

In [3]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/undergroundModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/undergroundModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'modelo_reblogeado.xlsx'

In [4]:
if undergroundDatabaseName == openPitDatabaseName:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = undergroundMineDataframe
else:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [12]:
import sys  
sys.path.insert(0, '../AuxiliarCodes/')
import gurobipy   as     gp
from   gurobipy   import GRB
from drawpointFunction  import drawpointFunction
from globalFunctions import getNumberOfBlocksInADimension
from itertools import chain
from functools import reduce


class MasterProblem:
    #Underground Model + Crown Pillar Restrictions.
    def __init__(self, database, numberOfPeriods, colHeight, minColHeight,pos_x,pos_y,pos_z,pos_x_f,pos_y_f):
        self.database = database
        self.numberOfPeriods = numberOfPeriods
        self.DP_init = 0       #### Tipo de extracción
        self.desc = 0.1
        self.colHeight = colHeight#630#300
        self.minColHeight = minColHeight#0.20
        self.pos_x = pos_x     
        self.pos_y = pos_y   
        self.pos_z = pos_z
        self.pos_x_f = pos_x_f     
        self.pos_y_f = pos_y_f  
        self.p_t = 3791.912
        self.orientationToExtractTheDrawpoints = 0

    def setParameters(self):
        self.getUndergroundVariablesFromCSV()
        self.getUndergroundInfo()
        self.setUndergroundParameters()
        self.setUndergroundMineLimits()
        self.setUndergroundVariables()
        #self.setVandB_vParameters()
    
    def getUndergroundVariablesFromCSV(self):
        self.undergroundBlocksLenght = self.database['X'].to_dict()             
        self.undergroundBlocksWidth  = self.database['Y'].to_dict()             
        self.undergroundBlocksHeight = self.database['Z'].to_dict()             
        self.undergroundBlockTonnage = self.database['Ton'].to_dict()              
        self.undergroundBlockMineral  = self.database['Mineral'].to_dict()          
        self.undergroundBlockRecovery  = self.database['Recuperación'].to_dict()     
        self.undergroundCopperLaw  = self.database['%Cu'].to_dict()
        self.undergroundExtractionFixedCosts = self.database['CPlanta CA'].to_dict()
        self.undergroundVariableExtractionCosts = self.database['CMina CA'].to_dict()
        self.undergroundCP_S = self.database['CPlanta S'].to_dict()
        self.undergroundCM_S = self.database['CMINA S'].to_dict() 
    
    def getUndergroundInfo(self):
        self.undergroundBlocks = [i for i in range(len(self.undergroundBlocksLenght.values()))]

    def setUndergroundParameters(self):
        #Underground Parameters
        self.t_S   = {period : period + 1 for period in range(self.numberOfPeriods)}
        self.MU_mt = {period : 25806600.0/1  for period in range(self.numberOfPeriods)} #Tonleage es mina
        self.ML_mt = {period : 0/3  for period in range(self.numberOfPeriods)}
        self.MU_pt = {period : 17777880.0/1  for period in range(self.numberOfPeriods)}#Mineral es planta
        self.ML_pt = {period : 0/3 for period in range(self.numberOfPeriods)}
        self.qU_dt = {period : 1 for period in range(self.numberOfPeriods)}
        self.qL_dt = {period : 0.0001 for period in range(self.numberOfPeriods)}
        self.A_d   = {period : 2 for period in range(self.numberOfPeriods)}
        self.NU_nt = {period : 59 for period in range(self.numberOfPeriods)} 
        self.NL_nt = {period : 0 for period in range(self.numberOfPeriods)}
        self.N_t   = {period : 59* (1 + period) for period in range(self.numberOfPeriods)}
        self.RL_dt = {period : 0.2 for period in range(self.numberOfPeriods)}
        self.RU_dt = {period : 1 for period in range(self.numberOfPeriods)}

    def setUndergroundMineLimits(self):
        self.undergroundBlocksLenghtLimits = getNumberOfBlocksInADimension(self.undergroundBlocksLenght)
        self.undergroundBlocksWidthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksWidth)
        self.undergroundBlocksHeightLimits = getNumberOfBlocksInADimension(self.undergroundBlocksHeight)

    def setUndergroundVariables(self):
        self.drawpoint, self.G_d, self.Q_d,self.q_d, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw, self.drawpoints_blocks = drawpointFunction(
                        self.pos_x, self.pos_y, self.pos_z, self.colHeight, self.DP_init, self.undergroundBlocksLenghtLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits, self.undergroundBlockTonnage, self.undergroundCP_S, self.undergroundCM_S, self.undergroundBlockMineral,
                        self.undergroundCopperLaw, self.pos_x_f, self.pos_y_f,self.orientationToExtractTheDrawpoints)
        self.predecessorDict = {}
        self.predecessorDict[0] = []
        self.predecessorDict[1] = [0]
        for i in range(1,len(self.predecessor)):
            if self.predecessor[i][0] not in self.predecessorDict.keys():
                self.predecessorDict[self.predecessor[i][0]] = []
            self.predecessorDict[self.predecessor[i][0]].append(self.predecessor[i][1])

    def setModelandGetResults(self):
        self.objValue, self.variableValues, self.runtime, self.gap = self.setUndergroundModel()

    def setModel(self):                
        self.undergroundModel = gp.Model(name = 'Modelo Integrado')
        self.undergroundModel.Params.TimeLimit = 3600
        self.undergroundModel.Params.OutputFlag = 0

        # Underground  Model

              #14. Naturaleza de las variables
        self.x_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="x_d")
        self.y_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.CONTINUOUS, name="y_d")
        self.z_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="z_d")

        #1. Restricción sobre la cantidad de tonelaje máxima y mínima a extraer en cada periodo.
        Ton_Up = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) <= self.MU_mt[ti] for ti in self.t_S),
                                            "Min_max")
        
        Ton_low = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.G_d[d] for d in self.drawpoint) >= self.ML_mt[ti] for ti in self.t_S),
                                            "Min_min")
        #2. Restricción sobre la cantidad de material máxima y mínima a procesar en cada periodo.
        Mat_Up = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) <= self.MU_pt[ti] for ti in self.t_S),
                                            "Mat_max")

        Mat_low = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) >= self.ML_pt[ti] for ti in self.t_S)
                                            , "Mat_min")

        
        #3. Rango de leyes máximas y mínimas a procesar
        GQC_low = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * self.y_dt[d, ti] for d in self.drawpoint) >=
                                self.qL_dt[ti] * gp.quicksum(self.G_d[d] * self.y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_low")
        
        GQC_Up = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * self.y_dt[d, ti] for d in self.drawpoint) <=
                                self.qU_dt[ti] * gp.quicksum(self.G_d[d] * self.y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_Up")

        #4. Todos los puntos de extracción deben ser iniciados en el largo de la extracción
        Drawp_init = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for ti in self.t_S) == 1 for d in self.drawpoint), "Drawp_init")

        #5. Los puntos de extracción deben ser activados al menos en el mismo periodo para que se inicie la extracción 
        Drawpextract_61 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, tau] for tau in range(ti+1)) >= self.z_dt[d, ti]  
                                            for d in self.drawpoint for ti in self.t_S), "Drawpextract_61")


        #6. Existe una cantidad máxima y mínima de drawpoints a abrir en cada periodo.
        Drawpextract_64_1 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[ti] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_1")
        Drawpextract_64_2 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) >= self.NL_nt[ti] for ti 
                                                        in self.t_S)
                                                        , "Drawpextract_64_2")

        #7. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        Drawpextract_65 = self.undergroundModel.addConstrs((gp.quicksum(self.z_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")


        #8. Si iniciamos la extracción de un drawpoint esta debe durar por su duraci ́on determinada.
        ## Un drawpoint solamente puede ser extraido por un preiodo pre determinado (A_d)
        Drawpextract_62 = self.undergroundModel.addConstrs((gp.quicksum(self.z_dt[d, ti] for ti in self.t_S)  <= self.A_d[ti]  for d in self.drawpoint
                                                    for ti in self.t_S), "Drawp_62")

        ## Una vez se inicia extrayendo de un drawpoint, se continua extrayendo sin interrupción
        Drawpextract_63 = self.undergroundModel.addConstrs((self.A_d[ti] *(self.z_dt[d, ti] - self.z_dt[d, ti+1]) 
                                            - gp.quicksum(self.z_dt[d, tau] for tau in range(ti+1)) <= 0 
                                            for d in self.drawpoint for ti in range(0,max(self.t_S))), "Drawpextract_63")

        #9. Relación de variables, el porcentaje a extraer es 0 si no se extra un drawpoint.
        Drawpextract_66 = self.undergroundModel.addConstrs((self.y_dt[d, ti] <= self.z_dt[d, ti] for d in self.drawpoint for ti in self.t_S),
                                                    "Drawpextract_66")

        #10. Existe una tasa m ́ınima de extracci ́on para cada drawpoint a extraer.
        Drawpextract_67_1 = self.undergroundModel.addConstrs((self.RL_dt[ti] * self.z_dt[d, ti]  <=  self.y_dt[d, ti] for d in self.drawpoint
                                                        for ti in self.t_S), "Drawpextract_67_1")

        #11. La altura a extraer debe ser mayor a una cantidad m ́ınima.
        rest_11 = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d,ti] for ti in self.t_S)>= self.minColHeight for d in self.drawpoint))

        #12. No podemos extraer más del 100 % de un drawpoint.
        Reserver_cnst = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint),
                                                    "Reserver_cnst")

        #13. Si se activa un drawpoint, se extrae en ese periodo
        rest_13 = self.undergroundModel.addConstrs(self.x_dt[d,ti] <= self.z_dt[d, ti] for d in self.drawpoint for ti in self.t_S)

        #14. Naturaleza de variables.

        #15. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        rest_15 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")
        
        #16. Restricción sobre el inicio de la extracci ́on de los drawpoints.


        alternative = self.undergroundModel.addConstrs(gp.quicksum(self.x_dt[a,s] for s in range(0,ti+1)) >= self.x_dt[d, ti] for d in self.drawpoint for ti in self.t_S for a in self.predecessorDict[d])
        #resta_prec = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[self.predecessor[l][0], m]*(max(self.t_S)-m+1) for m in self.t_S) <=
        #                            gp.quicksum(self.x_dt[self.predecessor[l][1], m]*(max(self.t_S)-m+1) for m in self.t_S)  
        #                            for l in range(len(self.predecessor))), "DP_Sup")

        
        #Conjuntos para el crown pillar

        #Restricciones del crown pillar
        #Variable 1 si y solo si el crown pillar esta ubicado en la elevaci ́on v, 0 en otro caso.
        test_w = {745: 0.0,
            785: 0.0,
            825: 0.0,
            865: 0.0,
            905: 0.0,
            945: 0.0,
            985: 1.0,
            1025: 0.0,
            1065: 0.0,
            1105: 0.0,
            1145: 0.0,
            1185: 0.0,
            1225: 0.0,
            1265: 0.0,
            1305: 0.0}

        test_u = {0: 0.0,
            1: 1.0,
            2: 1.0,
            3: 1.0,
            4: 1.0,
            5: 1.0,
            6: 1.0,
            7: 1.0,
            8: 1.0,
            9: 1.0}
        
        self.w_v = self.undergroundModel.addVars(self.V, vtype=GRB.BINARY, name="w")
        #self.w_v = self.undergroundModel.addVars(self.V,lb=test_w, ub=test_w ,vtype=GRB.BINARY, name="w")
        #alfa = gp.quicksum(self.y_dt[d, ti]*((((self.p_t * self.q_d[d] - self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        #((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)
        
        #self.alfa = self.undergroundModel.addVar(vtype =GRB.CONTINUOUS, lb = alfa, ub = alfa, name="alfa")
        
        
        self.theta = self.undergroundModel.addVar(vtype=GRB.CONTINUOUS,name="theta")
        self.u_t = self.undergroundModel.addVars(self.t_S, vtype=GRB.BINARY, name="u")
        #self.u_t = self.undergroundModel.addVars(self.t_S,lb=test_u, ub=test_u ,vtype=GRB.BINARY, name="u")


        ##Código experimental
        #natu = self.undergroundModel.addConstr(gp.quicksum(self.u_t[t] for t in self.t_S) <= 4)
        time_rest = self.undergroundModel.addConstrs(self.u_t[t-1]<= self.u_t[t] for t in range(1, len(self.t_S)))
        limit_time = self.undergroundModel.addConstrs(self.y_dt[d,t] <= self.u_t[t] for d in self.drawpoint for t in self.t_S)

        pillar_2 = self.undergroundModel.addConstrs(gp.quicksum(self.y_dt[d, ti] 
                                                        for ti in self.t_S) <= self.rho_v[v] * self.w_v[v] + (1 - self.w_v[v]) for v in self.V for d in self.drawpoint)
       
        pillar_3 = self.undergroundModel.addConstr(gp.quicksum(self.w_v[v] for v in self.V) == 1)

        theta_restriction_1 = self.undergroundModel.addConstr(-gp.GRB.INFINITY <= self.theta)
        theta_restriction_2 = self.undergroundModel.addConstr(self.theta <= 99999999999999)

         #Función objetivo
        self.undergroundObjectiveFunction = self.theta + gp.quicksum(self.y_dt[d, ti]*((((self.p_t * self.q_d[d] - self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        ((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)

        self.undergroundModel.setObjective(self.undergroundObjectiveFunction, GRB.MAXIMIZE)
        self.undergroundModel.Params.MIPGap = 0.05
        
    def optimizeModel(self):
        self.undergroundModel.optimize()
        self.lista_variable_Integrado = (self.undergroundModel.getAttr(GRB.Attr.X, self.undergroundModel.getVars()))
        solucion = self.undergroundModel.objVal
        runtime = self.undergroundModel.Runtime
        gap_f = self.undergroundModel.MIPGap
        estimatedW_v = {key:value.X for key,value in self.w_v.items()}
        estimatedU_t = {key:value.X for key,value in self.u_t.items()}
        self.selectedHeight = [key for key,value in estimatedW_v.items() if value > 0][0]
        self.selectedTime = [key for key,value in estimatedU_t.items() if value > 0][0]
        self.x_dt_values = self.undergroundModel.getAttr('X', self.x_dt)
        self.y_dt_values = self.undergroundModel.getAttr('X', self.y_dt)
        self.z_dt_values = self.undergroundModel.getAttr('X', self.z_dt)
        return estimatedW_v, estimatedU_t, self.theta



In [13]:
import sys  
sys.path.insert(0, '../AuxiliarCodes/')

from   gurobipy   import GRB
from globalFunctions import getNumberOfBlocksInADimension
from openPitFunctions import finalBlock
from itertools import chain
import re
import subprocess as sp


class SubProblem:
   def __init__(self, database, minHeightUnderground, maxHeightUnderground,numberOfPeriods, safetyLevel):
      self.database = database
      self.numberOfPeriods = numberOfPeriods
      self.minHeightUnderground = minHeightUnderground
      self.maxHeightUnderground = maxHeightUnderground
      self.safetyLevel = safetyLevel
      self.numberOfDestinations = 1
      self.basePrice = 3791.912
      self.desc = 0.1

   def setParameters(self):
      self.setOpenPitVariables()
      self.getOpenPitInfo()
      self.setOpenPitParameters()
      self.setOpenPitMineLimits()
      self.setPossibleHeights()
      self.setHeightSets()

   def execute(self, B_v, u_t, height,isFinalIteration = False):
      self.wHeight = height
      self.createOmpInput(B_v, u_t)
      return (self.executeOmp(isFinalIteration))

   def setOpenPitVariables(self):
      self.openPitBlocksLength = self.database['X'].to_dict() 
      self.openPitBlocksWidth = self.database['Y'].to_dict() 
      self.openPitBlocksHeight = self.database['Z'].to_dict() #Los bloques se orientan de abajo hacia arriba, el bloque 0 es el que esta más abajo, 784 bloques
      self.L_b = self.database['Ton'].to_dict() #openPitBlockTonnage
      self.o_b = self.database['Mineral'].to_dict() #openPitBlockMineral
      self.openPitBlockRecovery = self.database['Recuperación'].to_dict() #openPitBlockRecovery
      self.openPitCopperLaw = self.database['%Cu'].to_dict() #openPitCopperLaw
      self.c_pbt = self.database['CPlanta CA'].to_dict() #openPitPlantCapacity
      self.c_mbt = self.database['CMina CA'].to_dict() #openPitMineCapacity

   def getOpenPitInfo(self):
      self.openPitBlocks = [i for i in range(len(self.openPitBlocksLength.values()))]

   def setOpenPitParameters(self):
      #OpenPit Parameters
      self.t_C   = {period : period + 1 for period in range(self.numberOfPeriods)}
      self.RMu_t = {period : 2*25806600.0/1 for period in range(self.numberOfPeriods)}#Superior infinita, 0 por abajo Originales: 13219200
      self.RMl_t = {period : 0.0/3 for period in range(self.numberOfPeriods)}#Valor original 8812800.0
      self.RPu_t = {period : 2*17777880.0/1 for period in range(self.numberOfPeriods)}#Valor original 10933380.0
      self.RPl_t = {period : 0/3 for period in range(self.numberOfPeriods)}#Valor original 7288920.0 
      self.qu_t  = {period : 1 for period in range(self.numberOfPeriods)}#Leyes promedio maxima y minima.
      self.ql_t  = {period : 0.0001 for period in range(self.numberOfPeriods)}
      self.delta = {period: 0 for period in range(self.numberOfPeriods)}
      self.maxTimeOpenPit = self.t_C[max(self.t_C)]

   def setOpenPitMineLimits(self):
      self.openPitBlocksLengthLimits = getNumberOfBlocksInADimension(self.openPitBlocksLength)
      self.openPitBlocksWidthLimits = getNumberOfBlocksInADimension(self.openPitBlocksWidth)
      self.openPitBlocksHeightLimits = getNumberOfBlocksInADimension(self.openPitBlocksHeight)
      self.predecessorBlock = self.setPredecessorBlocks()

   def setPredecessorBlocks(self):
      self.predecessorBlocks = finalBlock(self.openPitBlocks, self.openPitBlocksLengthLimits,self.openPitBlocksWidthLimits, self.openPitBlocksHeightLimits)
  
   def setPossibleHeights(self):
      self.blockHeight, self.maxHeight, self.minHeight, self.numOfDifferentsBlocks = self.openPitBlocksHeightLimits
   
   def setHeightSets(self):
      #Acá hay que redifinir self.B_v para que tenga las alturas de maxheight al sumarle el safetylvl

      self.V = [height for height in chain(range(self.minHeight,self.maxHeight,self.blockHeight), [self.maxHeight])]
      self.B_v = {}
      self.rho_v = {v:( ((v- self.safetyLevel - self.minHeightUnderground)/(self.maxHeightUnderground - self.minHeightUnderground)) if v - self.minHeightUnderground > 0 else 0 ) for v in self.V}

      for v in self.V:
         numberOfBlocksBelowV = (self.openPitBlocksLengthLimits[3]*self.openPitBlocksWidthLimits[3])*((v-self.minHeight)/self.openPitBlocksHeightLimits[0])
         blocksBelowV = [block for block in range(int(numberOfBlocksBelowV)) if not numberOfBlocksBelowV == 0]
         self.B_v[v] = blocksBelowV
         
   def createOmpInput(self, infeasibleBlocks, u_t):
      self.writeProblemFile(u_t)
      self.writeBlocksFile(infeasibleBlocks)
      self.writePrecFile()
      self.writeParamsFile()
   
   def writeProblemFile(self, u_t):
         with open('../FilesToExecuteOmpOpenPit/files/openPit.prob', 'w') as f:
            numberOfDestinations = 'NDESTINATIONS: ' + str(self.numberOfDestinations)
            numberOfPeriods = 'NPERIODS: ' + str(self.numberOfPeriods)
            objective = 'OBJECTIVE: 0 1'
            duration = 'DURATION: 2'
            discountRate = 'DISCOUNT_RATE: '+ str(self.desc)#/self.numberOfPeriods)
            
            tonUpConstraint = 'CONSTRAINT: 0 3 P * L '
            for rmu in self.RMu_t.values():
               tonUpConstraint +=str(rmu) + " "

            tonLowContraint = 'CONSTRAINT: 1 3 P * G '
            for rml in self.RMl_t.values():
               tonLowContraint +=str(rml) + " "
            
            matUpConstraint = 'CONSTRAINT: 2 4 P * L '
            for rpu in self.RPu_t.values():
               matUpConstraint +=str(rpu) + " "

            matLowConstraint = 'CONSTRAINT: 3 4 P * G '
            for rpl in self.RPl_t.values():
               matLowConstraint +=str(rpl) + " "
            
            #las siguientes dos restricciones tienen 0 en vez de qut y qlt ya que pasamos la desigualdad hacia la derecha en las lineas 144 y 145
            copperLawUpConstraint = 'CONSTRAINT: 4 5 B * L ' 
            for qut in self.qu_t.values():
               copperLawUpConstraint += "0 "

            copperLawLowConstraint = 'CONSTRAINT: 5 6 B * G '
            for qlt in self.ql_t.values():
               copperLawLowConstraint +="0 "

            infeasibleBlocks = 'CONSTRAINT: 6 7 P * L '
            for delta in self.delta.values():
               infeasibleBlocks +=str(delta) + " "
            
            durationConstraint = 'CONSTRAINT: 7 2 P * L '
            for u in u_t.values():
               durationConstraint += str(len(self.openPitBlocks) * (1-u)) + " "
            
            constraints = [tonUpConstraint, tonLowContraint, matUpConstraint, matLowConstraint, copperLawUpConstraint,copperLawLowConstraint,infeasibleBlocks, durationConstraint]
            self.numberOfConstraints = len(constraints)
            self.nConstraints = 'NCONSTRAINTS: ' + str(self.numberOfConstraints)
            f.write('{}\n{}\n{}\n{}\n{}\n{}\n'.format(numberOfDestinations, numberOfPeriods, objective, duration, discountRate, self.nConstraints))
            f.write('{}\n{}\n{}\n{}\n{}\n{}\n{}\n{}\n'.format(*constraints))

   def writeBlocksFile(self, numberOfInvaiableBlocks):
      #print(f'Se optimizó el subproblema con {numberOfInvaiableBlocks} bloques infactibles')
      with open('../FilesToExecuteOmpOpenPit/files/openPit.blocks', 'w') as f:
         for block in self.openPitBlocks:
            index = block
            value = ((self.basePrice*self.openPitCopperLaw[block]-self.c_pbt[block])*self.o_b[block])-(self.c_mbt[block]*self.L_b[block])
            duration = 1 #Cuanto se demora en extraer el bloque
            ton = self.L_b[block]
            mineral = self.o_b[block]
            copperLawUpper = self.openPitCopperLaw[block] * self.L_b[block] - self.qu_t[0] * self.L_b[block]
            copperLawLower = self.openPitCopperLaw[block] * self.L_b[block] - self.ql_t[0] * self.L_b[block]

            #1 si no se puede extraer, 10977 última capa, los bloques van de abajo hacia arriba, 0 primer bloque de abajo, 10977 última capa hacia arriba, con 10975 la sol es vacia
            if block < numberOfInvaiableBlocks:
               f.write(('{} {} {} {} {} {} {} {}\n').format(index, value, duration, ton, mineral,copperLawUpper, copperLawLower, 1))
            else:
               f.write(('{} {} {} {} {} {} {} {}\n').format(index, value, duration, ton, mineral,copperLawUpper, copperLawLower, 0))
                 
   def writePrecFile(self):
      with open('../FilesToExecuteOmpOpenPit/files/openPit.prec', 'w') as f:
         for index,blockList in enumerate(self.predecessorBlocks):
            predecessorLine = str(len(blockList))
            for block in blockList:
               if block == index:
                  predecessorLine = " 0"
                  break
               else:
                  predecessorLine +=" " + str(block)
            f.write("{} {}\n".format(index, predecessorLine))
            
   def writeParamsFile(self):
      with open('../FilesToExecuteOmpOpenPit/params/openPit.params', 'w') as f:
         f.write("""USE_DISPLAY: 1
WRITE.LP.SOLUTION: 1
WRITE.IP.SOLUTION: 1
CPIT: 1
PP.ULTIMATE_PIT: 1
PP.FORCE_UPIT: 1
PP.EARLY_START: 1
PP.WASTE_OPTION: 1
PP.ELIM_NULL: 0
PP.TRANSITIVE_REDUCTION: 0
AG.USE_BLOCK_AGGREGATION: 0
AG.BLOCK_AGGREGATION: 0
OPTMETHOD: 0
CG.ONE_DESTINATION: 1
CG.IMPLICIT: 0
CG.USE_DISPLAY: 1
CG.MAX_ITER: -1
CG.TARGET_GAP: 0.0001
CG.MAX_TIME: -1
CG.USE_KSTEP: 0
CG.KSTEP_K: 10
CG.MASTER_NTHREADS: 4
CG.DISPLAY_DUALS: 1
HE.TOPOSORT: 1
HE.FTOPOSORT: 0
HE.NALPHA_POINTS: 50
HE.OPT_DESTINATIONS: 1
HE.NAIVE: 0
HE.NAIVE_INTSOLLIM: 1000
HE.NAIVE_EPGAP: 0.01
HE.NAIVE_TILIM: 14400
CP.DYNAMIC_CUTS: 0
CP.CLIQUES: 0
CP.MINW: 0
CP.DELAYED_PRECEDENCES: 0
CONSTRAINT_PROGRAMMING: 0
CPROG.GAP_LIMIT: 0.01
CPROG.CP_TIME_LIMIT: 28800
CPROG.EX_TIME_LIMIT: 28800
CPROG.HOT_START: 1
CPROG.NTHREADS: 8""")

   def executeOmp(self, isFinalIteration):
      output = sp.getoutput("./omp.sh ../FilesToExecuteOmpOpenPit/files/openPit.* ../FilesToExecuteOmpOpenPit/params/dbs_duals.params")
      #print(output)
      return self.getPiAndObjectiveValue(output, isFinalIteration)
   
   def getPiValues(self, output):
      pi_positions = [positions.start() for positions in re.finditer("rhs= 0.000000", output)]
      pi_t = dict.fromkeys(self.t_C,0)
      pi_indexes = [int(index) for index in range(self.numberOfPeriods * (self.numberOfConstraints -2), self.numberOfPeriods * (self.numberOfConstraints -1)) ]
      for pos in pi_positions:
         pi_value = float(output[pos-48: pos].split()[-3])
         pi_index = float(output[pos-48: pos].split()[-5])
         if pi_index in pi_indexes:
            period_index = pi_index - self.numberOfPeriods * (self.numberOfConstraints - 2)
            pi_t[period_index] = pi_value
      return pi_t
   
   def getSigmaValues(self, output):
      sigma_indexes = [index for index in range(self.numberOfPeriods * (self.numberOfConstraints -1), self.numberOfPeriods * (self.numberOfConstraints)) ]
      sigma_t = dict.fromkeys(self.t_C,0)
      for index in sigma_indexes:
         strToFind = "BZ_DUAL: row= "+ str(index)
         sigma_positions = [positions.start() for positions in  re.finditer(strToFind, output)]
         for pos in sigma_positions:
            #4print(output[pos: pos+48].split())
            sigma_index = float(output[pos: pos+48].split()[2])
            sigma_value = float(output[pos: pos+48].split()[4])
            period_index = sigma_index - self.numberOfPeriods * (self.numberOfConstraints - 1)
            sigma_t[period_index] = sigma_value
            #print(sigma_index,sigma_value)
      #print(sigma_t)
      return sigma_t
     
   def getPiAndObjectiveValue(self, output, isFinalIteration):
      objective_value_to_use = 0
      if isFinalIteration:
         objective_value_to_use = 1
         

      pi_t = self.getPiValues(output)
      self.pi_bDict = {}
      for v in self.V:
         for ti in self.t_C:
            if v == self.wHeight:
               self.pi_bDict[v,ti] = pi_t[ti]
            else:
               self.pi_bDict[v,ti] = 0
    
      sigma_t = self.getSigmaValues(output)
      objective_value_positions = [positions.start() for positions in re.finditer("Objval", output)]
      objective_value = (output[objective_value_positions[objective_value_to_use]: objective_value_positions[objective_value_to_use]+1000].split())
      indice_tot = objective_value.index("tot")
      objective_value = float(objective_value[indice_tot+2])
      
      return objective_value, self.pi_bDict , sigma_t 


In [22]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 10
        self.epsilon = 1000000
        self.safetyLevel = 60
        self.colHeight = 790#300 max 630 altura total de los drawpoints
        self.minColHeight = 0.40
        self.pos_x = 440#440#Coordenada x desde donde empezamos a extraer     
        self.pos_y = 550#550#Coordenada y desde donde empezamos a extraer
        self.pos_z = 530#780#Coordenada z desde donde empezamos a extraer     
        self.pos_x_f = 720#720#Coordenada x hazta donde extrameos  
        self.pos_y_f = 910#910#Coordenada y hazta donde extrameos
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.pos_z, self.pos_z + self.colHeight ,self.numberOfPeriods, self.safetyLevel)
        self.SubProblem.setParameters()

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods,self.colHeight,self.minColHeight ,self.pos_x, self.pos_y, self.pos_z, self.pos_x_f, self.pos_y_f)
        self.MasterProblem.setParameters()
        
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v
    
    """
    def calculateInvaiableBlocksWithSafetyLevel(self, selectedHeight):
        heightWithSafetyLevel = selectedHeight# + self.safetyLevel
        if heightWithSafetyLevel not in self.MasterProblem.B_v.keys():
            closestHeight = next((height for height in sorted(self.MasterProblem.V) if height >= heightWithSafetyLevel), None)
            if closestHeight == None:
                return len(self.SubProblem.openPitBlocks)
            return len(self.MasterProblem.B_v[closestHeight])
        return len(self.MasterProblem.B_v[heightWithSafetyLevel])
    """



    """
    def getResults(self):
        self.MasterProblem.setModel()
        model = self.MasterProblem.undergroundModel
        # Definir la referencia a MasterProblem y SubProblem
        #model._MasterProblem = self.MasterProblem.
        #model._SubProblem = self.SubProblem
        def benders_callback(model, where):
            #if where == GRB.Callback.MIPSOL:
                # Obtener la solución actual
                #w_v_sol = model.cbGetSolution(model._MasterProblem.w_v)
                #u_t_sol = model.cbGetSolution(model._MasterProblem.u_t)
                #theta_sol = model.cbGetSolution(model._MasterProblem.theta)
                w_v_sol,u_t_sol,theta_sol= self.MasterProblem.optimizeModel()
                print(w_v_sol,u_t_sol,theta_sol)
                # Resolver el subproblema dado la solución actual
                subproblem_obj_value, pi_vb, sigma_t = self.SubProblem.execute(len(self.MasterProblem.B_v[self.MasterProblem.selectedHeight]), u_t_sol, self.MasterProblem.selectedHeight, False)

                # Agregar la restricción de Benders
                model.cbLazy(self.MasterProblem.theta <= (subproblem_obj_value + gp.quicksum(gp.quicksum((w_v_sol[v] - self.MasterProblem.w_v[v]) * len(self.SubProblem.B_v[v]) * pi_vb[v,ti] for ti in self.SubProblem.t_C) for v in self.SubProblem.V) + gp.quicksum(len(self.SubProblem.openPitBlocks) * (u_t_sol[t] - self.MasterProblem.u_t[t]) * sigma_t[t] for t in self.MasterProblem.t_S)))

    # Crea
        # Definir el resto del modelo aquí

        # Establecer la función de callback
        self.MasterProblem.undergroundModel.Params.lazyConstraints = 1
        try:
            model.optimize(benders_callback)
            print(self.MasterProblem.undergroundModel)
        except gp.GurobiError as e:
            print('Error code ' + str(e.errno) + ': ' + str(e))
        except AttributeError:
            print('Encountered an attribute error')
        #print(self.MasterProblem.selectedHeight)
        #print(model._MasterProblem.selectedHeight)
    """
    def getResults(self):
        self.MasterProblem.setModel()


        def benders_callback(model, where):
                if where == GRB.Callback.MIPSOL:
                    print("entre")
                    # Obtener la solución actual
                    w_v_sol = model.cbGetSolution(self.MasterProblem.w_v)
                    u_t_sol = model.cbGetSolution(self.MasterProblem.u_t)
                    estimatedTheta = model.cbGetSolution(self.MasterProblem.theta)
                    self.MasterProblem.selectedHeight = [key for key,value in w_v_sol.items() if value > 0][0]

                    # Ejecuta tu implementación de Benders aquí y obten los valores de retorno
                    subproblem_obj_value, pi_vb, sigma_t = self.SubProblem.execute(len(self.MasterProblem.B_v[self.MasterProblem.selectedHeight]), u_t_sol, self.MasterProblem.selectedHeight, False)

                    # Agregar la restricción de Benders
                    model.cbLazy(self.MasterProblem.theta <= (subproblem_obj_value + gp.quicksum(gp.quicksum((w_v_sol[v] - self.MasterProblem.w_v[v]) * len(self.SubProblem.B_v[v]) * pi_vb[v,ti] for ti in self.SubProblem.t_C) for v in self.SubProblem.V) + gp.quicksum(len(self.SubProblem.openPitBlocks) * (u_t_sol[t] - self.MasterProblem.u_t[t]) * sigma_t[t] for t in self.MasterProblem.t_S)))     
                    print(w_v_sol, u_t_sol)
        self.MasterProblem.undergroundModel.params.Presolve = 0
        #self.MasterProblem.undergroundModel.params.PreCrush = 1
        self.MasterProblem.undergroundModel.Params.lazyConstraints = 1
        self.MasterProblem.undergroundModel.optimize(benders_callback)
        #print(self.MasterProblem.undergroundModel.ObjVal)



In [23]:
main2 = Main(undergroundMineDataframe, openPitDataframe)
main2.execute()

3391
Set parameter TimeLimit to value 3600
entre
{530: -0.0, 570: -0.0, 610: -0.0, 650: -0.0, 690: -0.0, 730: -0.0, 770: -0.0, 810: -0.0, 850: -0.0, 890: -0.0, 930: -0.0, 970: -0.0, 1010: -0.0, 1050: -0.0, 1090: -0.0, 1130: -0.0, 1170: -0.0, 1210: -0.0, 1250: -0.0, 1290: 1.0} {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0}


In [16]:
main2.MasterProblem.undergroundModel.ObjVal

1290159949.5361903